In [ ]:
# import pandas as pd
# from ast import literal_eval

# if not isinstance(df_reviews['category'].iloc[0], list):
#     df_reviews['category'] = df_reviews['category'].apply(lambda x: literal_eval(x) if pd.notna(x) else [])

# pharmacy_rows = df_reviews[df_reviews['category'].apply(lambda x: 'Hotel' in x)]

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import storage

project_id = "sharp-matter-449521-u2"
!gcloud config set project {project_id}

In [ ]:
!pip install pyspark py4j
!wget -P /usr/lib/spark/jars/ https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName("BigDataProcessing") \
    .config("spark.jars", "/usr/lib/spark/jars/gcs-connector-hadoop3-latest.jar") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true") \
    .getOrCreate()

In [ ]:
import pandas as pd
df_sample_pd = pd.read_parquet("gs://final_dataset_dat490/sample_reviews_stratified_10k.parquet")
df_sample_pd = df_sample_pd.sort_values(["gmap_id", "timestamp"]).reset_index(drop=True)
df_sample_pd.columns

In [ ]:
df_sample_pd['year'].unique()

In [ ]:
df_sample_pd.groupby('year')['gmap_id'].count()

In [ ]:
df_sample_pd.groupby('standard_category')['gmap_id'].count()

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
from transformers import pipeline
from tqdm import tqdm

tqdm.pandas()

# Zero-shot classification pipeline with BART
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

In [ ]:
candidate_labels = ["positive", "neutral", "negative"]

# Get the reviews as a list
texts = df_sample_pd["reviews"].tolist()

# Run batch inference
results = zero_shot_classifier(texts, candidate_labels)

# Extract top predicted labels
bart_labels = [res["labels"][0].capitalize() for res in results]

# Assign back to the DataFrame
df_sample_pd["bart_label"] = bart_labels


In [ ]:
df_sample_pd.head()

In [ ]:
df_sample_pd.columns

In [ ]:
!pip install --upgrade openai

In [ ]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get("openai_api_key"))

In [ ]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get("openai_api_key"))

models = client.models.list()

for model in models.data:
    print(model.id)

In [ ]:
from openai import OpenAI
from google.colab import userdata
from tqdm import tqdm
import json
import re
import time

# Set up client
client = OpenAI(api_key=userdata.get("openai_api_key"))

def extract_json_list(reply):
    reply = reply.strip().strip("```json").strip("```").strip()
    match = re.search(r'\[.*?\]', reply, re.DOTALL)
    if match:
        return json.loads(match.group())
    else:
        raise ValueError("No JSON list found in GPT reply")

# Input reviews
review_texts = df_sample_pd["reviews"].tolist()
batch_size = 5
all_labels = []

for i in tqdm(range(0, len(review_texts), batch_size)):
    batch = review_texts[i:i+batch_size]

    reviews_block = "\n".join([f"{j+1}. {text}" for j, text in enumerate(batch)])

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a helpful assistant that classifies customer reviews as Positive, Neutral, or Negative. "
                        "Only respond with a JSON list of labels."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"Please classify the following customer reviews:\n\n{reviews_block}\n\n"
                        "Respond with a JSON list like this: [\"Positive\", \"Neutral\", \"Negative\", ...]"
                    )
                }
            ],
            temperature=0,
            max_tokens=100
        )
        reply = response.choices[0].message.content.strip()
        labels = extract_json_list(reply)
        all_labels.extend(labels)

    except Exception as e:
        print(f"Error on batch {i//batch_size}: {e}")
        all_labels.extend([None] * len(batch))

    time.sleep(0.5)

In [ ]:
from openai import OpenAI
from google.colab import userdata
from google.colab import drive
import pandas as pd
from tqdm import tqdm
import json
import re
import time

drive.mount('/content/drive')

client = OpenAI(api_key=userdata.get("openai_api_key"))

def extract_json_list(reply):
    reply = reply.strip().strip("```json").strip("```").strip()
    match = re.search(r'\[.*?\]', reply, re.DOTALL)
    if match:
        return json.loads(match.group())
    else:
        raise ValueError("No JSON list found in GPT reply")

# Loading the Excel file from Google Drive
file_path = "/content/drive/MyDrive/Reviews std DAT490 labelled.xlsx"
df = pd.read_excel(file_path)

review_texts = df["Reviews"].head(30).tolist()
batch_size = 5
all_labels = []

for i in tqdm(range(0, len(review_texts), batch_size)):
    batch = review_texts[i:i+batch_size]

    reviews_block = "\n".join([f"{j+1}. {text}" for j, text in enumerate(batch)])

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a helpful assistant that classifies customer reviews as Positive, Neutral, or Negative. "
                        "Only respond with a JSON list of labels."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"Please classify the following customer reviews:\n\n{reviews_block}\n\n"
                        "Respond with a JSON list like this: [\"Positive\", \"Neutral\", \"Negative\", ...]"
                    )
                }
            ],
            temperature=0,
            max_tokens=100
        )
        reply = response.choices[0].message.content.strip()
        labels = extract_json_list(reply)
        all_labels.extend(labels)

    except Exception as e:
        print(f"Error on batch {i//batch_size}: {e}")
        all_labels.extend([None] * len(batch))

    time.sleep(0.5)

if "chatgpt_label" in df.columns:
    df.loc[:29, "chatgpt_label"] = all_labels # Apply labels to the 30 reviews we have labelled
else:
    labels_df = pd.DataFrame({'chatgpt_label': all_labels})
    df_labelled = pd.concat([df.head(30).reset_index(drop=True), labels_df], axis=1)

    df["chatgpt_label"] = None
    df.loc[:29, "chatgpt_label"] = all_labels

output_file_path = "/content/drive/MyDrive/Reviews std DAT490 labelled_labelled.xlsx"
df.to_excel(output_file_path, index=False)

print(f"Labels generated for the first 30 reviews and saved to: {output_file_path}")

In [ ]:
df_sample_pd["chatgpt_label"] = all_labels


In [ ]:
df_sample_pd.head()

In [ ]:
# df_sample_pd.to_parquet("gs://final_dataset_dat490/sample_reviews_stratified_10k_labeled.parquet")

In [ ]:
import pandas as pd
df_sample_lab = pd.read_parquet("gs://final_dataset_dat490/sample_reviews_stratified_10k_labeled.parquet")
df_sample_lab = df_sample_lab.sort_values(["gmap_id", "timestamp"]).reset_index(drop=True)
df_sample_lab.columns

In [ ]:
sentiment_map = {"Negative": 0, "Neutral": 1, "Positive": 2}

In [ ]:
df_sample_lab["chatgpt_label_encoded"] = df_sample_lab["chatgpt_label"].map(sentiment_map)

In [ ]:
df_yearly = (
    df_sample_lab
    .groupby(["business_name", "year"])
    .agg({
        "rating": "mean",
        "chatgpt_label_encoded": "mean",
        "num_of_reviews": "count",
        "avg_rating": "first",
        "state": "first",
        "standard_category": "first",
        "Monday": "first",
        "Tuesday": "first",
        "Wednesday": "first",
        "Thursday": "first",
        "Friday": "first",
        "Saturday": "first",
        "Sunday": "first"
    })
    .reset_index()
    .rename(columns={
        "rating": "avg_rating_year",
        "num_of_reviews": "review_count"
    })
)


In [ ]:
df_sample_pd['gmap_id'].nunique()

In [ ]:
df_sample_pd.groupby(["business_name", "year"])['rating'].mean()

In [ ]:
df_reviews_cleaned = pd.read_parquet("gs://final_dataset_dat490/dat490_final_dataset_cleaned.parquet")

In [ ]:
from pyspark.sql.functions import col, sum as spark_sum, when
from functools import reduce

weekday_cols = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

df_cleaned = df_reviews_cleaned.dropna(subset=weekday_cols)

from pyspark.sql.functions import first

df_year_status = (
    df_cleaned
    .groupBy("business_name", "year")
    .agg(*[first(col(day)).alias(day) for day in weekday_cols])
)

df_year_status = df_year_status.withColumn(
    "days_open", reduce(lambda a, b: a + b, [col(day) for day in weekday_cols])
).withColumn(
    "is_closed", when(col("days_open") == 0, 1).otherwise(0)
)

open_2019 = df_year_status.filter((col("year") == 2019) & (col("is_closed") == 0)).select("business_name")
closed_2020 = df_year_status.filter((col("year") == 2020) & (col("is_closed") == 1)).select("business_name")

from pyspark.sql.functions import broadcast

open_then_closed = open_2019.join(broadcast(closed_2020), on="business_name", how="inner")

open_then_closed.show(truncate=False)
print(f"Number of businesses open in 2019 and closed in 2020: {open_then_closed.count()}")


In [ ]:
open_2020 = df_year_status.filter(
    (col("year") == 2020) & (col("is_closed") == 0)
).select("business_name")

closed_2021 = df_year_status.filter(
    (col("year") == 2021) & (col("is_closed") == 1)
).select("business_name")

open_then_closed_2021 = open_2020.join(
    broadcast(closed_2021), on="business_name", how="inner"
)

open_then_closed_2021.show(truncate=False)
print(f"Number of businesses open in 2020 and closed in 2021: {open_then_closed_2021.count()}")

In [ ]:
df_sample_lab.groupby('year')['gmap_id'].count()

In [ ]:
df_sample_lab.columns

In [ ]:
df_filtered_years = df_sample_lab[df_sample_lab["year"].between(2018, 2021)]

business_years = df_filtered_years[["business_name", "year"]].drop_duplicates()

year_counts = business_years.groupby("business_name")["year"].nunique().reset_index()
year_counts = year_counts.rename(columns={"year": "year_count"})

businesses_all_4_years = year_counts[year_counts["year_count"] == 4]["business_name"]

df_4yr = df_filtered_years[df_filtered_years["business_name"].isin(businesses_all_4_years)]

print(f"Number of businesses with reviews in all 4 years (2018–2021): {businesses_all_4_years.nunique()}")
df_4yr[["business_name", "year"]].drop_duplicates().sort_values(["business_name", "year"]).head(10)


In [ ]:
weekday_cols = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

df_cleaned = df_reviews_cleaned.dropna(subset=weekday_cols)


In [ ]:
from pyspark.sql.functions import col, sum as spark_sum, when
from functools import reduce
df_filtered_years = df_cleaned.filter(col("year").between(2018, 2021))


In [ ]:
business_years = (
    df_filtered_years
    .select("business_name", "year")
    .dropDuplicates()
)


In [ ]:
from pyspark.sql.functions import count

year_counts = (
    business_years
    .groupBy("business_name")
    .agg(count("year").alias("year_count"))
)


In [ ]:
businesses_all_4_years = (
    year_counts
    .filter(col("year_count") == 4)
    .select("business_name")
)


In [ ]:
from pyspark.sql.functions import broadcast
df_4yr = (
    df_filtered_years
    .join(broadcast(businesses_all_4_years), on="business_name", how="inner")
)


In [ ]:
df_4yr.select("business_name", "year").dropDuplicates().orderBy("business_name", "year").show(20, truncate=False)
print(f"Number of businesses with reviews in all 4 years (2018–2021): {businesses_all_4_years.count()}")


In [ ]:
df_4yr.printSchema()

In [ ]:
df_4yr.count()

In [ ]:
df_4yr.select("business_name").distinct().count()

In [ ]:
# df_4yr.write.mode("overwrite").parquet("gs://final_dataset_dat490/engagement_parquet")


Fine-tuning BERT on ChatGPT-4o labels so that we can use that to label 658k rows for 5608 businesses which are open through all 4 years, from 2018 to 2021.

In [ ]:
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
df_sample_lab["label"] = df_sample_lab["chatgpt_label"].map(label_map)
df_sample_lab.head()

In [ ]:
from transformers import AutoTokenizer

# Choosing DistilBERT for less computational resources and more efficiency
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenization function
def tokenize_function(example):
    return tokenizer(example["reviews"], truncation=True, padding="max_length", max_length=256)


In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset
# Turning to huggingface dataset
hf_dataset = Dataset.from_pandas(df_sample_lab[["reviews", "label"]])


In [ ]:

tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)


In [ ]:
tokenized_dataset[0].keys()


In [ ]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro")
    }


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
)



In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [ ]:
!pip install wandb
import wandb
wandb.login()

In [ ]:
wandb.init(project="DAT490")

In [ ]:
trainer.train()


In [ ]:
from sklearn.metrics import classification_report

predictions = trainer.predict(tokenized_dataset["test"])
y_pred = predictions.predictions.argmax(-1)
y_true = predictions.label_ids

# Report
print(classification_report(y_true, y_pred, target_names=["Negative", "Neutral", "Positive"]))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
model.save_pretrained("/content/drive/MyDrive/bert_sentiment_model")
tokenizer.save_pretrained("/content/drive/MyDrive/bert_sentiment_model")


In [ ]:
df_4yr_pd = pd.read_parquet("gs://final_dataset_dat490/engagement_parquet")
df_4yr_pd = df_4yr_pd.sort_values(["business_name", "year"])

In [ ]:
model_path = "/content/drive/MyDrive/bert_sentiment_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()
# Checking for GPU availability in Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm.notebook import tqdm # For progress bar

MAX_LEN = 256

BATCH_SIZE = 128

class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_len):
        # Ensuring reviews are strings and handle potential NaN values
        self.reviews = [str(review) if pd.notna(review) else "" for review in reviews]
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = self.reviews[item]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

review_texts = df_4yr_pd['reviews'].tolist()

review_dataset = ReviewDataset(
    reviews=review_texts,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

review_data_loader = DataLoader(
    review_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, # No need to shuffle for inference
    num_workers=0
)

all_predictions = []
model.eval() # Ensuring model is in evaluation mode

print(f"Starting inference on {len(review_dataset)} reviews with batch size {BATCH_SIZE}...")

# Disable gradient calculations for inference to save memory and compute
with torch.no_grad():
    # Progress bar
    for batch in tqdm(review_data_loader, total=len(review_data_loader)):
        # Moving input tensors to the GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Logits are the raw scores output by the last layer
        logits = outputs.logits

        # torch.argmax finds the index of the maximum value along dimension 1 (the class dimension)
        preds = torch.argmax(logits, dim=1)

        # Moving predictions back to CPU and convert to numpy array
        all_predictions.extend(preds.cpu().numpy())

print(f"Inference complete. Obtained {len(all_predictions)} predictions.")


df_4yr_pd['label'] = all_predictions
print("Predictions added to the 'label' column in the DataFrame.")

print("\nDataFrame head with new labels:")
print(df_4yr_pd[['reviews', 'label']].head())

In [ ]:
df_4yr_pd.groupby('label')['gmap_id'].count()

In [ ]:
df_4yr_pd.to_parquet("gs://final_dataset_dat490/df_4yr_with_bert_labels.parquet")

In [ ]:
file_path = "/content/drive/MyDrive/Reviews std DAT490 labelled.xlsx"

df_reviews_30 = pd.read_excel(file_path)

In [ ]:
review_texts_30 = df_reviews_30['Reviews'].astype(str).tolist()

dataset_30 = ReviewDataset(
    reviews=review_texts_30,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

loader_30 = DataLoader(
    dataset_30,
    batch_size=8,  # Small batch is fine for 30 reviews
    shuffle=False
)


In [ ]:
model.eval()
predictions_30 = []

with torch.no_grad():
    for batch in loader_30:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        predictions_30.extend(preds.cpu().numpy())


In [ ]:
# Mapping numeric labels to sentiment
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

df_reviews_30["bert_label"] = [label_map[p] for p in predictions_30]

df_reviews_30[["Reviews", "bert_label"]].head()


In [ ]:
# df_reviews_30.to_excel("/content/drive/MyDrive/Reviews_30_with_BERT_labels.xlsx", index=False)


In [ ]:
df_4yr_pd = pd.read_parquet("gs://final_dataset_dat490/df_4yr_with_bert_labels.parquet")
df_4yr_pd = df_4yr_pd.sort_values(["business_name", "year"])

In [ ]:
df_4yr_pd.groupby('standard_category')['gmap_id'].count()

In [ ]:
import pandas as pd

# Copying the dataframe to avoid modifying original
df = df_4yr_pd.copy()

weekday_cols = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df["days_open"] = df[weekday_cols].sum(axis=1)

df_yearly = (
    df.groupby(["business_name", "year"])
    .agg({
        "rating": "mean",
        "label": "mean",
        "reviews": "count",
        "days_open": "mean",
        "state": "first",
        "standard_category": "first"
    })
    .reset_index()
    .rename(columns={
        "rating": "avg_yearly_rating",
        "label": "avg_sentiment",
        "reviews": "review_count",
        "days_open": "avg_days_open"
    })
)

df_yearly.head()


In [ ]:
from sklearn.preprocessing import MinMaxScaler

features = ["avg_yearly_rating", "avg_sentiment", "review_count", "avg_days_open"]

scaler = MinMaxScaler()
df_yearly[features] = scaler.fit_transform(df_yearly[features])
df_yearly.head()

In [ ]:
df_yearly["composite_score"] = df_yearly[features].sum(axis=1)
df_yearly.head()

In [ ]:
print("max:", round(df_yearly['composite_score'].max(), 2), "min:", (round(df_yearly['composite_score'].min(), 2)))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(df_yearly["composite_score"], kde=True)
plt.title("Distribution of Composite Score")
plt.show()


The distribution of composite score is skewed to the left. This means that we have more businesses with higher composite score. We must address this imbalance while using the model.

In [ ]:
# Pivoting composite_score
pivot = df_yearly.pivot(index='business_name', columns='year', values='composite_score')
pivot.columns = [f"year_{col}" for col in pivot.columns]

pivot = pivot.dropna(subset=["year_2018", "year_2019", "year_2020", "year_2021"])

category_df = df_yearly[df_yearly["year"] == 2021][["business_name", "standard_category"]].drop_duplicates()

# Merging category back into pivot
pivot = pivot.merge(category_df, on="business_name", how="left")

# Change in composite score - patterns in one year gap
pivot["delta_18_19"] = pivot["year_2019"] - pivot["year_2018"]
pivot["delta_19_20"] = pivot["year_2020"] - pivot["year_2019"]
pivot["pct_change_20_21"] = ((pivot["year_2021"] - pivot["year_2020"]) / pivot["year_2020"]) * 100

# Engagement metrics
def classify_change(pct):
    if pct <= -20:
        return 0
    elif pct >= 20:
        return 2
    else:
        return 1

pivot["engagement_label"] = pivot["pct_change_20_21"].apply(classify_change)

X = pivot[["delta_18_19", "delta_19_20", "standard_category"]]
X = pd.get_dummies(X, columns=["standard_category"])
y = pivot["engagement_label"].values


In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

# train+val and test (80/20)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Split train+val into train and val (75/25 of the 80% => 60/20 overall)
X_train_full, X_val, y_train_full, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42
)

print(f"Train: {X_train_full.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}")


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_sample_weight


In [ ]:
def get_stratified_subset(X, y, fraction, random_state=42):
    splitter = StratifiedShuffleSplit(n_splits=1, train_size=fraction, random_state=random_state)
    train_idx, test_idx = next(splitter.split(X, y))

    if isinstance(X, pd.DataFrame):
        return X.iloc[train_idx], y[train_idx]
    else:
        return X[train_idx], y[train_idx]


In [ ]:
def train_xgb_subset(X_train_full, y_train_full, train_fraction, X_val, y_val):
    X_train, y_train = get_stratified_subset(X_train_full, y_train_full, train_fraction)

    sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

    model = XGBClassifier(
        objective='multi:softmax',
        num_class=3,
        eval_metric='mlogloss',
        random_state=42
    )
    model.fit(X_train, y_train, sample_weight=sample_weights)

    preds = model.predict(X_val)
    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro')
    cm = confusion_matrix(y_val, preds)
    auc = roc_auc_score(y_val, model.predict_proba(X_val), multi_class='ovr')
    rep = classification_report(y_val, preds)

    return acc, f1, model


In [ ]:
train_sizes = [0.2, 0.4, 0.6, 0.8, 0.99]
xgb_accuracies = []
xgb_f1s = []

X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

for frac in train_sizes:
    acc, f1, model = train_xgb_subset(X_train_full, y_train_full, frac, X_val, y_val)
    xgb_accuracies.append(acc)
    xgb_f1s.append(f1)
    print(f"Train size: {int(frac*100)}% — Accuracy: {acc:.4f} — F1-score: {f1:.4f}")


In [ ]:
import matplotlib.pyplot as plt

x_vals = [int(f * 100) for f in train_sizes]

plt.figure(figsize=(8, 5))
plt.plot(x_vals, xgb_accuracies, marker='o', label="XGBoost Accuracy")
plt.plot(x_vals, xgb_f1s, marker='s', label="XGBoost F1-score")
plt.xlabel("Training Set Size (%)")
plt.ylabel("Score")
plt.title("XGBoost Performance vs. Training Set Size")
plt.legend()
plt.ylim(0, 1)
plt.grid(True)
plt.show()


In [ ]:
final_preds = model.predict(X_test)

print(f"Test Set, Accuracy: {acc:.4f} — F1-score: {f1:.4f}")

In [ ]:
print(classification_report(y_test, final_preds))

In [ ]:
cm = confusion_matrix(y_test, final_preds)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


In [ ]:
df_10k = pd.read_parquet("gs://final_dataset_dat490/sample_reviews_stratified_10k_labeled.parquet")
df_10k.columns

In [ ]:
sns.set(style="whitegrid")

label_counts = df_10k['chatgpt_label'].value_counts().reindex(['Positive', 'Neutral', 'Negative'])

plot_df = pd.DataFrame({
    'Sentiment': label_counts.index,
    'Count': label_counts.values,
    'Method': 'ChatGPT'
})

plt.figure(figsize=(8, 5))
sns.barplot(data=plot_df, x='Method', y='Count', hue='Sentiment', palette={
    'Positive': '#4CAF50', 'Neutral': '#FFC107', 'Negative': '#F44336'
})

plt.title("Sentiment Label Distribution (ChatGPT - 10,000 Reviews)")
plt.ylabel("Number of Reviews")
plt.xlabel("Sentiment Analysis Method")
plt.legend(title='Sentiment')
plt.tight_layout()
plt.show()